In [ ]:
import pandas as pd

link = r'C:\Projects\geochem_python_lessons\Lessons\4 Distribution\Lesson_4.gdb\urals_N41_O41'
cols = [i.name for i in arcpy.ListFields(link) if i.name not in ['Shape']]
data = [i for i in arcpy.da.SearchCursor(link, cols)]

df = pd.DataFrame(data, columns = cols).set_index('OBJECTID')

df

In [ ]:
# выберем названия столбцов для анализа
cols_sub = cols[1:-2]
cols_sub

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import os

# Функция рисует две гистограммы для нормальных данных и логнормальные данные
def hist_plots(data, element_name, filepath):
    
    # функция подписи результатов теста хи-квадрат
    def chi_square_text(data, ax):
        stat, p = stats.chisquare(data)
        string = f"Chi-Square test\nstats: {stat:.3f}\np-value: {p:.3f}"
        ax.text(0.95, 0.95, string, horizontalalignment='right',
                 verticalalignment='top', transform= ax.transAxes)
    
    # создадим объект графика (размер в дюймах)
    plt.figure(figsize=(16/2.54, 12/2.54), layout = 'constrained')
    ax1 = plt.subplot(2,1,1)
    ax2 = plt.subplot(2,1,2)
    
    # отрисуем нормальные данные
    sns.histplot(data, stat = 'probability', log_scale = False, ax=ax1)
    
    # добавим подпись результата теста Хи-Квадрат (верхний график)
    chi_square_text(data, ax1)
    
    # отрисуем логнормальные данные
    #sns.histplot(data, stat = 'probability', log_scale = True, ax=ax2)
    sns.ecdfplot(data, log_scale = True, ax=ax2)
    
    # добавим подпись результата теста Хи-Квадрат (нижний график)
    chi_square_text(np.log(np.array(data)), ax2)
    
    # сохраним график в файл
    plt.savefig(os.path.join(filepath, f'{element_name}_hist.png'))
    
    # Покажем график
    plt.show()

# применим функцию
filepath = r'C:\Projects\geochem_python_lessons\Lessons\4 Distribution'
col = 'CU'
hist_plots(data = df[col], element_name = col, filepath = filepath)

In [ ]:
# График эмпирической кумулятивной функции распределения
sns.ecdfplot(data, log_scale = True)
plt.show()

In [ ]:
# График плотности распределения
sns.kdeplot(data, log_scale = True)
plt.show()

In [ ]:
# Q-Q график
def qq_plot(data):
    
    # отсортируем в вариационный ряд
    data = np.sort(data)
    
    # наблюдаемые квантили
    quantiles_observed = stats.zscore(data)
    
    # теоретические квантили
    quantiles_theoretical = np.sort(np.random.normal(0, 1, len(data)))
    
    # график
    plt.scatter(quantiles_observed, quantiles_theoretical)
    
    # линия равенства xy
    xpoints = ypoints = quantiles_theoretical
    plt.plot(xpoints, ypoints, linestyle='--', color='r', lw=2, scalex=False, scaley=False)
    
    # оси и вывод
    plt.xlabel('Observed quantiles')
    plt.ylabel('Theoretical quantiles')
    plt.show()

qq_plot(np.log(np.array(data)))

In [ ]:
# сделаем график по группам

# сводная группа
df['Groups'] = df['Source'] + ' ' + df['Landscape']

def get_quantiles(df, column, groups, log_scale):
    # отсортируем значения
    df = df.loc[:, [column, groups]].sort_values(by = [groups, column])
    # log
    if log_scale:
        df[column] = np.log( df[column].to_numpy() )
    # вычислим квантили
    for group in df[groups].unique():
        idx = df[groups] == group
        data = df.loc[idx, column]
        df.loc[idx, 'Observed quantiles'] = stats.zscore(data)
        df.loc[idx, 'Theoretical quantiles'] = np.sort(np.random.normal(0, 1, len(data)))
    
    return df

# объявим функцию для рисования графиков
def dist_plot_by_groups(df, column, groups, log_scale = True): 
    
    # создадим объект графика
    plt.figure(figsize = (16/2.54, 16/2.54), layout = 'tight')
    ax1 = plt.subplot(2,2,1)
    ax2 = plt.subplot(2,2,2)
    ax3 = plt.subplot(2,1,2)
    
    # ECFD
    sns.ecdfplot(data = df, x = column, hue = groups, log_scale = log_scale, ax = ax1)
    
    # Q-Q график
    df_quantiles = get_quantiles(df, column, groups, log_scale = log_scale)
    sns.scatterplot(data = df_quantiles, x = 'Observed quantiles', 
                    y = 'Theoretical quantiles', hue = groups, ax = ax2)
    # линия равенства x-y
    xpoints = ypoints = df_quantiles['Theoretical quantiles']
    ax2.plot(xpoints, ypoints, ls = '--', color = 'r', lw = 2, scalex = False, scaley = False)
    
    #  Гистограмма
    #sns.histplot(data = df, x = column, hue = groups, stat = 'probability', log_scale = log_scale, ax = ax3)
    sns.kdeplot(data = df, x = column, hue = groups, log_scale = log_scale, common_norm = False, ax = ax3)
    
    # покажем график
    plt.show()

dist_plot_by_groups(df, 'CU', 'Groups')

In [ ]:
# приведение содержания хим.элементов к референс-группе

# выберем референс-группу
main_group = 'O41 frame'

# функция для приведения содержания к референ-группе
def equalize_values(df, columns, groups, main_group):
    # составим список остальных групп
    other_groups = [i for i in df[groups].unique() if i != main_group]

    # сделаем копию df, в которую будем вставлять изменённые значения
    df_modified = df.copy()

    # приведём значения в цикле по столбцам
    for col in columns:
        # вычислим статистические параметры референс-группы
        main_idx = df[groups] == main_group
        main_data = df.loc[main_idx, col].to_numpy()
        main_data_log = np.log(main_data)
        main_mean = np.mean(main_data_log)
        main_std = np.std(main_data_log)
    
        # обработаем данные по группам
        for group in other_groups:
            # вычислим логарифмы
            idx = df[groups] == group
            data = df.loc[idx, col].to_numpy()
            data_log = np.log(data)
    
            # стандартизируем
            zscores = stats.zscore(data_log)
    
            # приведём логарифмы к референс-группе
            data_modified_log = (zscores * main_std) + main_mean
    
            # вернём к истинным значениям
            data_modified = np.exp(data_modified_log)
    
            # запишем в таблицу
            df_modified.loc[idx, col] = data_modified
    
    return df_modified

df_modified = equalize_values(df, 'Groups', main_group)
dist_plot_by_groups(df_modified, 'CU', 'Groups')